---

# CSCI 3202, Fall 2023
# Homework 5.1
# Due: 15th November at 11:59 pm

<br> 

### Your name: Owen Kutzscher

<br> 

# Mancala Game Implementation

In this assignment, you are tasked with implementing various functions for a Mancala game. The game is played on a board with specific rules, and you will need to implement the core game logic by completing the `play`, `valid_move`, and `winning_eval` functions. You are provided with the `init` and `display_board` functions. The assignment is divided into two parts:

## Mancala rules to be followed 
**(there are few modifications from the original game, please read this before writing the code)**

- On every turn, select a pit and distribute its stones in an anti-clockwise direction.
    - If the last stone lands in the player's mancala, in an opponent's pit, or in one of the player's non-empty pits, no further action is taken, and the current player's turn ends.
    - If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side has some stones, collect all those stones, including the one that just landed, and place them into the current player's mancala.

- If any of the player's pits are entirely empty, the game concludes. The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.

## Part 1: Small Board (3 Pits of 2 Stones each)

For the first part of the assignment, students will work on a small Mancala board. The board consists of 3 pits, each initially containing 2 stones. The students need to implement the following:

1. **play**: Implement the `play` function to allow players to take turns and make moves. The function should correctly distribute stones according to the specified game rules. The game should also switch between players after each play.

2. **valid_move**: Implement the `valid_move` function to ensure that a player's chosen move is valid. It should check if the selected pit is not empty and falls within the allowed pit range.

3. **winning_eval**: Implement the `winning_eval` function to determine when the game is over and which player wins. The game ends when any player's pits are all empty. The winner is the player with the most stones in their mancala. If both mancalas have the same number of stones, it's a tie.

Students should test their code by playing a sequence of moves starting with Player 1: 1, 2, 3, 2, 1. \
So, it would be P1 picks pit 1, P2 picks pit 2, P1 picks pit 3...and so on.
The pits are 1-indexed when displaying and picking to make a move.

Pick the pit irrespective of its validity, and print invalid move message if chosen pit is empty or invalid.

The output generated by this experiment must match the expected output given below.

## Part 2: Play Against a Random Player (6 Pits of 4 Stones each)

In the second part of the assignment, students will extend their implementation to a larger board. The board consists of 6 pits with 4 stones in each pit. In addition to the `play`, `valid_move`, and `winning_eval` functions, students need to create a random move generator for a random player. This random player selects a random valid pit with stones to make a move. The following steps are involved in creating the random move generator:

1. **Random Move Generator**: Define the `random_move_generator` that selects a random pit from the available non-empty pits for the random player. The random player should choose a move based on these criteria. \
Set the 'seed' value to ensure that the generated values remain consistent and reproducible when grading.

You may refer to these links: [How to generate random integers in Python](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/#:~:text=Random%20integer%20values%20can%20be,for%20the%20generated%20integer%20values.), [How to use seed in Python random](https://www.w3schools.com/python/ref_random_seed.asp)


The objective is to play up to **10** moves in total (5 moves by student, 5 moves by random player), allowing the students to verify whether their code correctly implements the Mancala game logic. 

The output submitted should reflect the state of the board and the moves played.

**Please make sure to call the `display_board` function after each move for both the parts and run all the cells before submitting**

In [1]:
import random

In [12]:
class Mancala:
    def __init__(self, pits_per_player=3, stones_per_pit = 2):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        # print(self.current_player)
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        pit_index = 0
        if self.current_player == 1:
            pit_index = pit - 1
        else:
            pit_index = pit + self.pits_per_player
            
        # make sure pit is within bounds
        if (0 <= pit_index) & (pit < len(self.board)): 
            # print('na bitch you aint in the bounds n shit')
            # make sure pit is not a player's mancala
            if (pit_index != self.pits_per_player) & (pit_index != (self.pits_per_player*2)+1):
                # print('naaaaaaaaaaa thats the other players mancala')
                # make sure pit has stones in it
                if self.board[pit_index] != 0:
                    # print('FUUUUUUUCCCCCCCCKKKKKKKKK')
                    # add the move to the list of valid moves
                    move = self.current_player, pit
                    self.moves.append(move)
                    return True                    
        return False
        
        pass
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        
        random.seed(69)
        random_move = random.randint(1, self.pits_per_player)
        
        while self.valid_move(random_move) == False:
            random_move = random.randint(1, self.pits_per_player)
        
        return random_move

        
        
        
        # write your code here
        pass
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        
        # write your code here
        
        print("Player", self.current_player,"chose pit:", pit)
        
        # 1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        if self.valid_move(pit) == False:
            print("INVALID MOVE")
            return self.board
        
        
        # 2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        if self.winning_eval() == True:
            print("GAME OVER")
            return self.board
        
        
        # 3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.
        
        # init variables
        pit_start = 0
        other_player_mancala = 0
        board_size = len(self.board)
        current_index = 0
        empty_pit_stones = 0
        
        
        
        # update the pit index depending on the player
        # pit index will be used to access the pit of choice and move the stones
        
        if self.current_player == 1:
            pit_start = pit
            other_player_mancala_index = self.pits_per_player*2 + 1
            current_player_mancala_index = self.pits_per_player
        else:
            pit_start = pit + 1 + self.pits_per_player
            other_player_mancala_index = self.pits_per_player
            current_player_mancala_index = self.pits_per_player*2 + 1
            
        
        
        # grab stones from specified pit
        stones = self.board[pit_start-1]
        self.board[pit_start-1] = 0
        
        
        # distribute stones across the board, skipping over the other player's mancala
        i=0
        
        while i<stones:
            current_index = (i+pit_start)%board_size
            if current_index == other_player_mancala_index:
                stones += 1
            else:
                self.board[current_index] += 1
            i += 1        

        
        # check if the last stone was dropped into an empty pit on players side
        # if so take all stones from this current pit and the opposite pit
        adjacent_index = abs(current_index - (current_index-self.pits_per_player)*2)
        # make sure pit is not a player's mancala
        if (current_index != self.pits_per_player) & (current_index != ((self.pits_per_player*2)+1)):
            if self.board[current_index] == 1:
                # make sure that we are on the current player's side
                
                if self.current_player == 1 & current_index in self.p1_pits_index:
                    self.board[current_player_mancala_index] += self.board[current_index]+self.board[adjacent_index]
                    self.board[current_index] = 0
                    self.board[adjacent_index] = 0
                if self.current_player == 2 & current_index in self.p1_pits_index:
                    self.board[current_player_mancala_index] += self.board[current_index]+self.board[adjacent_index]
                    self.board[current_index] = 0
                    self.board[adjacent_index] = 0
                
        
        
        
        # Finally, the function then switches the current player, allowing the other player to take their turn.
        if self.current_player == 1:
            self.current_player = 2
        else:
            self.current_player = 1
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        winning_state = True
        # check player 1's side
        for i in range(0, self.pits_per_player):
            if self.board[i] != 0:
                winning_state = False
        if winning_state == True:
            return True
        
        winning_state = True
        
        # check player 2's side
        for i in range(self.pits_per_player+1, self.pits_per_player*2+1):
            if self.board[i] != 0:
                winning_state = False
        if winning_state == True:
            return True
        
        return False


In [14]:

# Mancala part 1 
game = Mancala()
game.display_board()

# Player 1 selects pit 1 (1-based index)
game.play(1)
game.display_board()

# Player 2 selects pit 2
game.play(2)
game.display_board()

# Player 1 selects pit 3
game.play(3)
game.display_board()

# Player 2 selects pit 2
game.play(2)
game.display_board()

# Player 1 selects pit 1
game.play(1)
game.display_board()

game.display_board()

# Printing the list of moves
print("\nList of valid moves:")
for move in game.moves:
    player, pit = move
    print(f"Player {player} selected pit {pit}")


P1               P2
     ____0____     
1 -> | 2 | 2 | <- 3
2 -> | 2 | 2 | <- 2
3 -> |_2_|_2_| <- 1
         0         
Turn: P1
False
Player 1 chose pit: 1
P1               P2
     ____0____     
1 -> | 0 | 2 | <- 3
2 -> | 3 | 2 | <- 2
3 -> |_3_|_2_| <- 1
         0         
Turn: P2
Player 2 chose pit: 2
P1               P2
     ____1____     
1 -> | 0 | 3 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_3_|_2_| <- 1
         0         
Turn: P1
Player 1 chose pit: 3
P1               P2
     ____1____     
1 -> | 0 | 3 | <- 3
2 -> | 3 | 1 | <- 2
3 -> |_0_|_3_| <- 1
         1         
Turn: P2
Player 2 chose pit: 2
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_0_|_3_| <- 1
         1         
Turn: P1
Player 1 chose pit: 1
INVALID MOVE
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_0_|_3_| <- 1
         1         
Turn: P1
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_0_|_3_| <-

#### Expected output for part 1

In [4]:
# P1               P2
#      ____0____     
# 1 -> | 2 | 2 | <- 3
# 2 -> | 2 | 2 | <- 2
# 3 -> |_2_|_2_| <- 1
#          0         
# Turn: P1
# Player 1 chose pit: 1

# P1               P2
#      ____0____     
# 1 -> | 0 | 2 | <- 3
# 2 -> | 3 | 2 | <- 2
# 3 -> |_3_|_2_| <- 1
#          0         
# Turn: P2
# Player 2 chose pit: 2

# P1               P2
#      ____1____     
# 1 -> | 0 | 3 | <- 3
# 2 -> | 3 | 0 | <- 2
# 3 -> |_3_|_2_| <- 1
#          0         
# Turn: P1
# Player 1 chose pit: 3

# P1               P2
#      ____1____     
# 1 -> | 0 | 3 | <- 3
# 2 -> | 3 | 1 | <- 2
# 3 -> |_0_|_3_| <- 1
#          1         
# Turn: P2
# Player 2 chose pit: 2

# P1               P2
#      ____1____     
# 1 -> | 0 | 4 | <- 3
# 2 -> | 3 | 0 | <- 2
# 3 -> |_0_|_3_| <- 1
#          1         
# Turn: P1
# Player 1 chose pit: 1
# Invalid move

# P1               P2
#      ____1____     
# 1 -> | 0 | 4 | <- 3
# 2 -> | 3 | 0 | <- 2
# 3 -> |_0_|_3_| <- 1
#          1         
# Turn: P1

# List of valid moves:
# Player 1 selected pit 1
# Player 2 selected pit 2
# Player 1 selected pit 3
# Player 2 selected pit 2

In [5]:
# Mancala part 2

In [6]:
game_2 = Mancala(6, 4)
game_2.display_board()

game_2.play(3)
game_2.display_board()

game_2.play(game_2.random_move_generator())
game_2.display_board()

game_2.play(3)
game_2.display_board()

game_2.play(game_2.random_move_generator())
game_2.display_board()

game_2.play(2)
game_2.display_board()

game_2.play(game_2.random_move_generator())
game_2.display_board()

game_2.play(3)
game_2.display_board()

game_2.play(game_2.random_move_generator())
game_2.display_board()

game_2.play(1)
game_2.display_board()

game_2.play(game_2.random_move_generator())
game_2.display_board()



P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
Player 1 chose pit: 3
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 0 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P2
Player 2 chose pit: 6
P1               P2
     ____1____     
1 -> | 5 | 0 | <- 6
2 -> | 5 | 4 | <- 5
3 -> | 1 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P1
Player 1 chose pit: 3
P1               P2
     ____1____     
1 -> | 5 | 0 | <- 6
2 -> | 5 | 4 | <- 5
3 -> | 0 | 4 | <- 4
4 -> | 6 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P2
Player 2 chose pit: 1
P1               P2
     ____1____     
1 -> | 5 | 0 | <- 6
2 -> | 5 | 5 | <- 5
3 -> | 0 | 5 | <- 4
4 -> | 6 | 5 | <- 3
5 -> | 5 | 5 | <- 2
6 -> |_5_|_0_| <